In [8]:
pip install scikit-learn nltk matplotlib pandas


  Using cached nltk-3.9.1-py3-none-any.whl.metadata (2.9 kB)
  Using cached joblib-1.5.2-py3-none-any.whl.metadata (5.6 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
  Using cached click-8.3.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached pyparsing-3.2.5-py3-none-any.whl.metadata (5.0 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   ---------------------------------------- 0.0/8.9 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.9 MB ? eta -:--:--
   - -------------------------------------- 0.3/8.9 MB ? eta -:--:--
   -- -

In [10]:
!python -m ipykernel install --user --name tfenv --display-name "Python (tfenv)"


Installed kernelspec tfenv in C:\Users\ASUS\AppData\Roaming\jupyter\kernels\tfenv


In [12]:
import sys
import tensorflow as tf
import numpy as np
import sklearn
import nltk
import matplotlib
import pandas as pd

# Print versions
print("Python:", sys.version)
print("TensorFlow:", tf.__version__)
print("NumPy:", np.__version__)
print("scikit-learn:", sklearn.__version__)
print("NLTK:", nltk.__version__)
print("Matplotlib:", matplotlib.__version__)
print("Pandas:", pd.__version__)


Python: 3.10.18 | packaged by Anaconda, Inc. | (main, Jun  5 2025, 13:08:55) [MSC v.1929 64 bit (AMD64)]
TensorFlow: 2.12.0
NumPy: 1.23.5
scikit-learn: 1.7.2
NLTK: 3.9.1
Matplotlib: 3.10.6
Pandas: 2.3.2


In [13]:
# Test imports for your project
import tensorflow as tf
import numpy as np
import sklearn
import nltk
import matplotlib.pyplot as plt
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

print("All imports successful! You're ready to run your AI/ML project.")


All imports successful! You're ready to run your AI/ML project.


In [14]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding

print("All libraries imported successfully.")


All libraries imported successfully.


In [18]:
import pandas as pd

# Tiny sample dataset
data = {
    "English": ["Hello", "How are you?", "I love you", "Good morning", "Thank you"],
    "French": ["Bonjour", "Comment ça va?", "Je t'aime", "Bonjour", "Merci"]
}

df = pd.DataFrame(data)
df.to_csv("eng-fra.csv", index=False)  # saves CSV in current folder
df.head()


,English,French
0,Hello,Bonjour
1,How are you?,Comment ça va?
2,I love you,Je t'aime
3,Good morning,Bonjour
4,Thank you,Merci


In [19]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

# Split English and French sentences
english_sentences = df['English'].values
french_sentences = df['French'].values

# Add <start> and <end> tokens for French sentences
french_sentences = ['<start> ' + sent + ' <end>' for sent in french_sentences]

# Tokenizer for English
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(english_sentences)
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_sequences = eng_tokenizer.texts_to_sequences(english_sentences)
eng_padded = pad_sequences(eng_sequences, padding='post')

# Tokenizer for French
fra_tokenizer = Tokenizer(filters='')
fra_tokenizer.fit_on_texts(french_sentences)
fra_vocab_size = len(fra_tokenizer.word_index) + 1
fra_sequences = fra_tokenizer.texts_to_sequences(french_sentences)
fra_padded = pad_sequences(fra_sequences, padding='post')

# Maximum lengths
max_eng_len = eng_padded.shape[1]
max_fra_len = fra_padded.shape[1]

print("English vocab size:", eng_vocab_size, "Max length:", max_eng_len)
print("French vocab size:", fra_vocab_size, "Max length:", max_fra_len)


English vocab size: 10 Max length: 3
French vocab size: 10 Max length: 5


In [20]:
# Decoder input is French sentence without last token
decoder_input = fra_padded[:, :-1]

# Decoder output is French sentence shifted by one (teacher forcing)
decoder_output = fra_padded[:, 1:]

# One-hot encode decoder output
decoder_output = tf.keras.utils.to_categorical(decoder_output, num_classes=fra_vocab_size)

# Split data into training and testing sets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test, dec_in_train, dec_in_test = train_test_split(
    eng_padded, decoder_output, decoder_input, test_size=0.2
)

print("Training samples:", X_train.shape[0])


Training samples: 4


In [21]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Embedding, Attention

embedding_dim = 64
latent_dim = 64

# Encoder
encoder_inputs = Input(shape=(max_eng_len,))
enc_emb = Embedding(eng_vocab_size, embedding_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(enc_emb)

# Decoder
decoder_inputs = Input(shape=(max_fra_len-1,))
dec_emb_layer = Embedding(fra_vocab_size, embedding_dim, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=[state_h, state_c])

# Attention
attention = Attention()
context_vector = attention([decoder_outputs, encoder_outputs])
decoder_concat_input = tf.keras.layers.Concatenate(axis=-1)([decoder_outputs, context_vector])

# Dense layer
decoder_dense = Dense(fra_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_concat_input)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 3)]          0           []                               
                                                                                                  
 input_2 (InputLayer)           [(None, 4)]          0           []                               
                                                                                                  
 embedding (Embedding)          (None, 3, 64)        640         ['input_1[0][0]']                
                                                                                                  
 embedding_1 (Embedding)        (None, 4, 64)        640         ['input_2[0][0]']                
                                                                                              

In [22]:
history = model.fit(
    [X_train, dec_in_train], y_train,
    epochs=200,
    batch_size=2,
    validation_split=0.2
)


Epoch 1/200
2/2 [==============================] - 6s 1s/step - loss: 2.3037 - accuracy: 0.0909 - val_loss: 2.2970 - val_accuracy: 0.3333
Epoch 2/200
2/2 [==============================] - 0s 24ms/step - loss: 2.2940 - accuracy: 0.2727 - val_loss: 2.2915 - val_accuracy: 0.6667
Epoch 3/200
2/2 [==============================] - 0s 26ms/step - loss: 2.2835 - accuracy: 0.5455 - val_loss: 2.2867 - val_accuracy: 0.6667
Epoch 4/200
2/2 [==============================] - 0s 27ms/step - loss: 2.2749 - accuracy: 0.5455 - val_loss: 2.2812 - val_accuracy: 0.6667
Epoch 5/200
2/2 [==============================] - 0s 27ms/step - loss: 2.2649 - accuracy: 0.4545 - val_loss: 2.2753 - val_accuracy: 0.6667
Epoch 6/200
2/2 [==============================] - 0s 27ms/step - loss: 2.2535 - accuracy: 0.4545 - val_loss: 2.2689 - val_accuracy: 0.3333
Epoch 7/200
2/2 [==============================] - 0s 27ms/step - loss: 2.2419 - accuracy: 0.4545 - val_loss: 2.2614 - val_accuracy: 0.3333
Epoch 8/200
2/2 [=====

In [6]:
from keras.models import load_model
import pickle

# Save the model
model.save(r"C:\Users\ASUS\Downloads\Seq2Seq Translator\eng_fra_translator.h5")

# Save tokenizers
with open(r"C:\Users\ASUS\Downloads\Seq2Seq Translator\english_tokenizer.pkl", 'wb') as f:
    pickle.dump(eng_tokenizer, f)

with open(r"C:\Users\ASUS\Downloads\Seq2Seq Translator\french_tokenizer.pkl", 'wb') as f:
    pickle.dump(fra_tokenizer, f)


NameError: name 'model' is not defined

In [7]:

history = model.fit([X_train, dec_in_train], y_train, epochs=200, batch_size=2, validation_split=0.2)


NameError: name 'model' is not defined

In [8]:
from keras.models import load_model
import pickle

# Load the trained model
model = load_model('eng_fra_translator.h5')

# Load tokenizers
with open('english_tokenizer.pkl', 'rb') as f:
    eng_tokenizer = pickle.load(f)

with open('french_tokenizer.pkl', 'rb') as f:
    fra_tokenizer = pickle.load(f)


OSError: No file or directory found at eng_fra_translator.h5

In [9]:
import os
os.getcwd()


'c:\\Users\\ASUS\\Downloads\\Seq2Seq translator'

In [10]:
from keras.models import load_model
import pickle
import os

folder_path = r"C:\Users\ASUS\Downloads\Seq2Seq Translator"

# Load model
model = load_model(os.path.join(folder_path, 'eng_fra_translator.h5'))

# Load tokenizers
with open(os.path.join(folder_path, 'english_tokenizer.pkl'), 'rb') as f:
    eng_tokenizer = pickle.load(f)

with open(os.path.join(folder_path, 'french_tokenizer.pkl'), 'rb') as f:
    fra_tokenizer = pickle.load(f)


OSError: No file or directory found at C:\Users\ASUS\Downloads\Seq2Seq Translator\eng_fra_translator.h5

In [11]:
from keras.models import load_model
import pickle
import os

folder_path = r'c:\Users\ASUS\Downloads\Seq2Seq translator'

# Load the trained model
model = load_model(os.path.join(folder_path, 'eng_fra_translator.h5'))

# Load the tokenizers
with open(os.path.join(folder_path, 'english_tokenizer.pkl'), 'rb') as f:
    eng_tokenizer = pickle.load(f)

with open(os.path.join(folder_path, 'french_tokenizer.pkl'), 'rb') as f:
    fra_tokenizer = pickle.load(f)


OSError: No file or directory found at c:\Users\ASUS\Downloads\Seq2Seq translator\eng_fra_translator.h5